In [1]:
import pandas as pd
import numpy as np
import gensim
import random
import sys
import glob
import os
import datetime
from nltk import sent_tokenize
from nltk import word_tokenize
from scipy.spatial.distance import cosine
import warnings
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string
from gensim.utils import simple_preprocess
from gensim.models.callbacks import CallbackAny2Vec
from itertools import product
from sklearn.metrics import precision_recall_curve

warnings.simplefilter('ignore')

sys.path.append("../../oats")
from oats.annotation.ontology import Ontology
from oats.distances import pairwise as pw
from oats.utils.utils import flatten

In [2]:
class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epochs = []
        self.epoch = 1
        self.losses = []
        self.deltas = []
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 1:
            delta = loss
        else:
            delta = loss- self.loss_previous_step
        self.loss_previous_step=loss
        self.losses.append(loss)
        self.epochs.append(self.epoch)
        self.epoch += 1
        self.deltas.append(delta)

In [3]:
model = gensim.models.Word2Vec.load("../models/plants_sg/saved/word2vec_both_dim200_500_a_s_min3_window8.model")

In [31]:
w = preprocess_string("cuticle ")[0]
model.most_similar(w,topn=5)

[('cutin', 0.6633152961730957),
 ('cuticular', 0.6314857006072998),
 ('wax', 0.5339478850364685),
 ('elast', 0.4423104524612427),
 ('vlc', 0.44112345576286316)]

In [9]:
model_wiki = gensim.models.Word2Vec.load("../models/wiki_sg/word2vec.bin")

In [32]:
w = "cuticle"
model_wiki.most_similar(w,topn=6)

[('epidermis', 0.7717389464378357),
 ('hyphae', 0.7492595911026001),
 ('pores', 0.7176331281661987),
 ('cuticular', 0.7021838426589966),
 ('integument', 0.6915901899337769),
 ('cuticles', 0.6817958950996399)]